

---
# 난독화 복원 솔루션: 싱글 모델



In [1]:
# =============================================================================
# 0. 환경 설정 (라이브러리 설치 및 드라이브 마운트)
# =============================================================================
!pip install transformers

from google.colab import drive
drive.mount('/content/drive')

import time
import re
import torch
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# =============================================================================
# 1. 경로 설정 (학습 완료된 모델 경로)
# =============================================================================
# [Signal] 분류기: 입력이 오타인지 야민정음인지 판단
PATH_CLASSIFIER = "/content/drive/MyDrive/P01_Nandoc/Classifier"

# [Core] 파이널 모델: 태그에 따라 복원을 수행하는 만능 모델
PATH_FINAL_MODEL = "/content/drive/MyDrive/P01_Nandoc/FINALV0"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Inference Device: {device}")

# =============================================================================
# 2. 통합 복원 시스템 클래스
# =============================================================================
class NandocSystem:
    def __init__(self):
        print("🔄 [System] 엔진 시동 중... (Intelligent Mode)")
        start_time = time.time()

        # A. 분류기 로드 (KoELECTRA)
        try:
            self.cls_tokenizer = ElectraTokenizer.from_pretrained(PATH_CLASSIFIER)
            self.cls_model = ElectraForSequenceClassification.from_pretrained(PATH_CLASSIFIER).to(device)
            self.cls_model.eval()
            print("✅ [1/2] 분류기(Signal) 장착 완료")
        except Exception as e:
            print(f"❌ 분류기 로드 실패: {e}")
            raise

        # B. 파이널 모델 로드 (KoBART)
        try:
            self.tokenizer = PreTrainedTokenizerFast.from_pretrained(PATH_FINAL_MODEL)
            self.model = BartForConditionalGeneration.from_pretrained(PATH_FINAL_MODEL).to(device)
            self.model.eval()
            print("✅ [2/2] 파이널 모델(Core) 장착 완료")
        except Exception as e:
            print(f"❌ 파이널 모델 로드 실패: {e}")
            raise

        print(f"🎉 [System] 시스템 준비 완료 ({time.time() - start_time:.2f}s)\n")

    # -------------------------------------------------------------------------
    # 🔒 방어 로직 (이중 잠금 + 정제) - 뇌절 원천 차단
    # -------------------------------------------------------------------------
    def _double_lock_cut(self, input_text, pred_text):
        # 1차: 문장 개수 동기화
        input_punctuations = len(re.findall(r'[.!?]', input_text))
        target_count = max(input_punctuations, 1)

        matches = [m.start() for m in re.finditer(r'[.!?]', pred_text)]
        if len(matches) >= target_count:
            cut_idx = matches[target_count - 1]
            candidate = pred_text[:cut_idx+1]
        else:
            candidate = pred_text

        # 2차: 길이 제한 (1.3배)
        limit_len = int(len(input_text) * 1.3) + 10
        if len(candidate) > limit_len:
            truncated = candidate[:limit_len]
            last_punc = max(truncated.rfind('.'), truncated.rfind('!'), truncated.rfind('?'))
            if last_punc != -1: return truncated[:last_punc+1]
            last_space = truncated.rfind(' ')
            if last_space != -1: return truncated[:last_space]
            return truncated
        return candidate

    def _final_polish(self, text):
        # 특수문자 제거 & 중복 문장 제거
        clean_pattern = r'[^가-힣a-zA-Z0-9\s.,?!\'"%~]'
        text = re.sub(clean_pattern, '', text)
        text = re.sub(r'\s+', ' ', text).strip()

        sentences = re.split(r'(?<=[.?!])\s+', text)
        unique_sentences = []
        for s in sentences:
            s = s.strip()
            if not s: continue
            if unique_sentences and s == unique_sentences[-1]:
                continue
            unique_sentences.append(s)
        return ' '.join(unique_sentences)

    # -------------------------------------------------------------------------
    # 🚀 메인 추론 함수 (Restore)
    # -------------------------------------------------------------------------
    def restore(self, text):
        if not text: return ""

        # Step 1. 분류 (Classifier) -> 상황 판단
        cls_inputs = self.cls_tokenizer(
            text, return_tensors='pt', truncation=True, max_length=128
        ).to(device)

        with torch.no_grad():
            outputs = self.cls_model(**cls_inputs)
            # 0: 오타/일반, 1: 야민정음
            class_id = torch.argmax(outputs.logits, dim=-1).item()
            probs = torch.softmax(outputs.logits, dim=-1)[0]
            confidence = probs[class_id].item()

        # Step 2. 태그 부착 (Tagging) -> 작업 지시
        # 학습 데이터와 동일한 태그를 붙여야 모델이 알아듣습니다.
        if class_id == 0:
            tagged_text = f"<오류> {text}"   # 오타 모드 발동
            info = "TYPE-0 (오타)"
        else:
            tagged_text = f"<노이즈> {text}" # 야민정음 모드 발동
            info = "TYPE-1 (야민)"

        # Step 3. 복원 (Generation) -> 파이널 모델 수행
        gen_inputs = self.tokenizer(
            tagged_text, return_tensors='pt', padding=True, truncation=True, max_length=128
        ).to(device)

        with torch.no_grad():
            gen_ids = self.model.generate(
                gen_inputs['input_ids'],
                attention_mask=gen_inputs['attention_mask'],
                num_beams=5,             # 정확도 향상
                no_repeat_ngram_size=3,  # 반복 방지
                repetition_penalty=1.5,  # 뇌절 억제
                early_stopping=True,
                max_length=256
            )
        raw_output = self.tokenizer.decode(gen_ids[0], skip_special_tokens=True)

        # Step 4. 방어 (Defense) -> 결과물 다듬기
        # 주의: 길이 제한은 태그 없는 '원본 text' 기준으로!
        cut_text = self._double_lock_cut(text, raw_output)
        final_text = self._final_polish(cut_text)

        return final_text, info, confidence


Mounted at /content/drive
🚀 Inference Device: cuda


In [ ]:

# =============================================================================
# 3. 실행 테스트
# =============================================================================
if __name__ == "__main__":
    # 1. 시스템 초기화
    nandoc = NandocSystem()

    print("\n" + "="*60)
    print("💡 [Nandoc Intelligent System] 가동 (종료: 'q')")
    print("="*60)

    # 2. 테스트 루프
    while True:
        user_input = input("\n📝 입력: ")
        if user_input.lower() in ['q', 'quit']:
            print("👋 종료합니다.")
            break

        if not user_input.strip(): continue

        # 복원 수행
        result, type_info, conf = nandoc.restore(user_input)

        print(f"   └─ [{type_info} | {conf:.0%}] -> {result}")

In [5]:
# =============================================================================
# 3. 5개 문장 자동 테스트 실행
# =============================================================================
if __name__ == "__main__":
    # 1. 시스템 초기화
    nandoc = NandocSystem()

    # 2. 테스트 문장 리스트 (오타, 야민정음, 혼합)
    test_sentences = [
        "아... 까껵 초코 쀼됴 뻥 뜛려셔 시원하자만 담배 냄쌢 밁버림. 싸게 하루만 묵껬댜! 하는 샤람한테만 츄천. 땸빼 냄새까 모튼 짱졈을 카쳝갸는 꼿. 놃래빵에써 캭종 댬빼와 유흐에 쪌였을 때 나는 냄새까 곘쑉 뱌에 이씀 ㅆ... 싸니까 할 먈 없음.",
        "찌안 츅쳐늚료 따너왔는떼 꺄쎵빠 쬬교 깔끔햐네요. 쩌렴햔뗖 물콰 음료똦 았꾜 빵뚀 아쮼 땨뜻했슴나땾. 휘낙쓰밙그와 껴러뚀 멸저 않얆서 쬬햤꾦용. 뺘롤 엿베 펴늬쩌멃 어쎠쎠 라했씀나땨. 땮으메또 아옹할 쓬 악쓰면 엋용햐꾜 쌉네요. 옊아까 어렵찌먄 ^^",
        "호텔은 깔끔하고 서Bs도 좋았는데, bangOl 작Oㅏ 창문e 겨익 없어서 환71㉠r 안 되는 느낌0I 8었어yO. 그2Hㅌ Oㅏ침 싀ㅆ는 신선ㅎr그 맛있었으ni 만족^러웠습L|ㅁ-.",
        "01번 즉말O|| 친ヲ랑 go서 1박 2일했어요. bangOl 깔끔ㅎrヱ b㉰ 전망0l 경말 멋졌어8. 조싀E 간단Ha겨만 맛있ヱ, 직원분dl0l 친절hri서 분읶71E 좋았죠. ㅁ-만, 수영장 입7㉠r 조금 빽빽ㅎri서 조금 71㉰렸=데, 그 외O||는 완전 만족s러운 oㅕ행0I었어yO.",
        "펴난하 쨜 쓀꼬 왔씀낭땿. 쭈위에 먓쩝뚔 많아쎠 쬬흔 켯 꺄탸요."
    ]

    print("\n" + "="*80)
    print("📊 [Nandoc Intelligent System] 5개 문장 복원 테스트")
    print("="*80)

    for i, text in enumerate(test_sentences):
        # 복원 수행
        result, type_info, conf = nandoc.restore(text)

        print(f"\n[Test Case {i+1}]")
        print(f"🕵️‍♂️ 감지된 타입: {type_info} (확률: {conf:.2%})")
        print(f"📝 원본: {text[:60]}..." if len(text) > 60 else f"📝 원본: {text}")
        print(f"✨ 복원: {result}")
        print("-" * 80)


🔄 [System] 엔진 시동 중... (Intelligent Mode)
✅ [1/2] 분류기(Signal) 장착 완료


You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


✅ [2/2] 파이널 모델(Core) 장착 완료
🎉 [System] 시스템 준비 완료 (0.71s)


📊 [Nandoc Intelligent System] 5개 문장 복원 테스트

[Test Case 1]
🕵️‍♂️ 감지된 타입: TYPE-0 (오타) (확률: 99.98%)
📝 원본: 아... 까껵 초코 쀼됴 뻥 뜛려셔 시원하자만 담배 냄쌢 밁버림. 싸게 하루만 묵껬댜! 하는 샤람한테만 츄천...
✨ 복원: 아... 가격 좋고 뷰도 뻥 뚫려서 시원하지만 담배 냄새 미쳐버림. 싸게 하루만 묵겠다! 하는 사람한테만 추천. 담배 냄새가 모든 장점을 가져가는 곳. 노래방에서 각종 담배와 유흥에 쩔었을 때 나는 냄새가 계속 방에 있음 ... 싸니까 할 말 없음.
--------------------------------------------------------------------------------

[Test Case 2]
🕵️‍♂️ 감지된 타입: TYPE-0 (오타) (확률: 99.98%)
📝 원본: 찌안 츅쳐늚료 따너왔는떼 꺄쎵빠 쬬교 깔끔햐네요. 쩌렴햔뗖 물콰 음료똦 았꾜 빵뚀 아쮼 땨뜻했슴나땾. 휘낙쓰...
✨ 복원: 지인 추천으로 다녀왔는데 가성비 좋고 깔끔하네요. 저렴한데 물과 음료도 있고 방도 있어 따뜻했습니다. 휘닉스파크와 거리도 멀지 않아서 좋았어요. 바로 옆에 편의점도 있어서 편리했습니다. 다음에도 이용할 수 있으면 이용하고 싶네요.
--------------------------------------------------------------------------------

[Test Case 3]
🕵️‍♂️ 감지된 타입: TYPE-1 (야민) (확률: 99.98%)
📝 원본: 호텔은 깔끔하고 서Bs도 좋았는데, bangOl 작Oㅏ 창문e 겨익 없어서 환71㉠r 안 되는 느낌0I 8었...
✨ 복원: 호텔은 깔끔하고 서비스도 좋았는데, 방이 작아 창문이 거의 없어서 환기가 안 되는 느낌이 들었어요. 그래도 아침 식사는 신선하고 맛있었으니 